# Лабораторная работа №1
## Поэлементные преобразования изображений

1. Считать цветное rgb изображение

2. Преобразовать изображение в градации серого

3. Написать функцию реализации препарирования изображения

4. По гистограмме изображения определить значение порога яркостей, обеспечивающего оптимальное разделение объекта и фона. Осуществить пороговую обработку входного изображения с найденным пороговым значением

5. Сделать пороговую обработку методом Otsu (Функция OpenCV)

6. Определить динамический диапазон входного изображения. Осуществить линейное контрастирование входного изображения в заданный динамический диапазон яркостей

7. Сделать эквализацию гистограммы изображения

8. Сделать эквализацию методом CLAHE (Функция OpenCV)

9. Осуществить препарирование изображения с заданной препарирующей функцией

*Для работы с изображением использовать OpenCV. Все необходимые для задания функции реализовавать самим, а не использовать готовые в OpenCV, если не указано обратного. После преобразований изображений выводить их гистограммы.*